# This is a scratch space for building functions
After functions are made, move them to the corresponding folder. 

In [1]:
from pathlib import Path
from flow_analysis_comps.PIV.PIV_process import AMF_PIV
from flow_analysis_comps.PIV.definitions import PIV_params, segmentMode
%matplotlib widget

In [2]:
data_adr = Path(r"U:\test_data\20250122_Plate017")
plate_id = r"20250122_Plate017"
video_id = r"031"
frame_id1 = r"Img_00000.tif"
frame_id2 = r"Img_00001.tif"

mode = "BRIGHTFIELD"
# mode = "FLUO"

fps = 20
winsize = 40 # pixels, interrogation window size in frame A
searchsize = 50  # pixels, search area size in frame B
overlap = 30 # pixels
frame_ids = (0,1)

STN_thresh = 1.05

piv_param_obj = PIV_params(
    video_path=str(data_adr/video_id / "Img"),
    segment_mode=segmentMode.BRIGHT,
    fps=fps,
    window_size=winsize,
    search_size=searchsize,
    overlap_size=overlap,
    stn_threshold=STN_thresh,
    px_per_mm= 1500/2
)

In [ ]:
amf_piv_obj = AMF_PIV(piv_param_obj)
amf_piv_obj.plot_raw_images(frame_ids)
amf_piv_obj.plot_segmentation()

In [ ]:
# amf_piv_obj.piv_process(frame_ids, FAKE_OUTLIERS=False)
amf_piv_obj.piv_process_windef(frame_ids)

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

um_per_px = 0.138
fps = 20

import_path = Path(r"U:\test_data\20250122_Plate017\031\OpenPIV_results_16_PIV_output\field_A0008.txt")
import_db = pd.read_table(import_path)
import_db = import_db[import_db['mask'] == 0]
import_db['abs'] = np.sqrt(import_db['u']**2 + import_db['v']**2)
import_db['um/s'] = import_db['abs'] * um_per_px * fps
import_db['dir'] = np.arctan2(import_db['v'], import_db['u'])

import_db = import_db[abs(import_db['dir']) - 0.4 < 0]

fig, ax = plt.subplots(2)
ax[0].hist(import_db['um/s'], bins=50, range=(1, 10))
ax[1].hist(import_db['dir'], bins=50, range=(-np.pi, np.pi))
import_db